# Apply Modelling to Generate Alternatives to a PyPSA-Sec Model Instance of Germany

In [ ]:
import pypsa
import pandas as pd
idx = pd.IndexSlice
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import pyomo.environ as pe
import logging
logger = logging.getLogger()
logger.setLevel(level=logging.INFO)
from scripts.prepare_model import (make_options, 
                                   extra_functionality,
                                   prepare_costs, 
                                   annuity)

from scripts.prepare_mga import make_mga_weights



pypsa.Network.lopf_prepare_solver = pypsa.opf.network_lopf_prepare_solver
pypsa.Network.lopf_solve_wo_build = pypsa.opf.network_lopf_solve

options = make_options()

In [ ]:
#options

# Export-import time series for the simplified representation of europe

In [ ]:
export_import_load = pd.read_csv("electricity_import_export_target2050_weather2011.csv", squeeze=True) * -1 # export - import

# Heat pump cop data

* https://github.com/oruhnau/when2heat/   
* https://www.nature.com/articles/s41597-019-0199-y, https://doi.org/10.1038/s41597-019-0199-y
* https://data.open-power-system-data.org/when2heat/2019-08-06

In [ ]:
try:
    when2heat = pd.read_csv("when2heat_DE_2011.csv", index_col = "utc_timestamp", parse_dates=True)
except FileNotFoundError:
    print("create subset for cop time series")
    # https://github.com/oruhnau/when2heat/   
    # https://www.nature.com/articles/s41597-019-0199-y
    when2heat = pd.read_csv("../../1/6Data/Heat/when2heat_stacked.csv", index_col = "utc_timestamp", parse_dates=True)
    when2heat = when2heat.loc[when2heat["country"]=="DE"].loc['2011-01-01 00:00:00':'2011-12-31 23:00:00']
    when2heat.to_csv("when2heat_DE_2011.csv")
    
def cop_DE(df, floor, radiator, water): 
    """extract ASHP and GSHP cop. Actually it is the Seasonal Performance Factor (SPF)"""
    
    if floor+radiator+water!=1:
        raise ValueError("floor+radiator+water must be 1")
    
    rows = when2heat.attribute.isin(["ASHP_floor", "ASHP_radiator", "ASHP_water", "GSHP_floor", "GSHP_radiator", "GSHP_water"])

    cop = when2heat.loc[rows].reset_index()
    cop = cop.pivot(index='utc_timestamp', columns = 'attribute', values = 'data')

    cop_agg = pd.DataFrame()
    cop_agg["ASHP"] = (cop["ASHP_floor"].values * floor 
                        + cop["ASHP_radiator"] * radiator 
                        + cop["ASHP_water"] * water)
    
    cop_agg["GSHP"] = (cop["GSHP_floor"].values * floor 
                        + cop["GSHP_radiator"] * radiator 
                        + cop["GSHP_water"] * water)
    # COP to Seasonal Performance Factor (SPF/ in German: JAZ), source:  Feldtest des Fraunhofer ISE
    # NB: correct with basic amount instead of multiplication
    spf_agg = cop_agg - cop_agg.sum() * (1-0.9)/cop_agg.shape[0]   
    return spf_agg
    
cop_DE = cop_DE(df=when2heat, floor=0.8*0.50, radiator=0.8*0.50, water=0.2)

# Günther et al. 20% water heating,  80% space heating: 85% with floor heating and 15% with radiators ()
# floor=0.8*0.85, radiator=0.8*0.15, water=0.2

del when2heat

In [ ]:
cop_DE.mean()

In [ ]:
es = pypsa.Network()

es.import_from_hdf5("all_flex-central_0_DE_1h.h5")

In [ ]:
# fig = plt.figure(figsize=(16, 10))
# fig.add_subplot(211)
# es.links_t.efficiency["DE central heat pump"].plot(ylim=(1.5, 4.5))
# fig.add_subplot(212)
# cop_DE["ASHP"].plot(ylim=(1.5, 4.5))'

In [ ]:
costs = prepare_costs(file_name = "pypsa-eur-sec-30/data/costs/costs.csv", number_years=1, usd_to_eur=1/1.2, costs_year=2030)

# Add solar-rooftop and load shedding

In [ ]:
es.add("Generator",
         "DE solar-rooftop",
         bus="DE",
         p_nom_extendable=True,
         carrier="solar",
         p_nom_max=es.generators.loc["DE solar","p_nom_max"],
         capital_cost = costs.at[idx['solar-rooftop', 2030],'fixed'],
         p_max_pu=es.generators_t["p_max_pu"].loc[:, "DE solar"],
         marginal_cost=costs.at[idx['solar', 2030],'VOM'])

es.add("Generator", "DE load shedding",
          bus="DE",
          p_nom_extendable=True,
          marginal_cost=1000.)

# es.add("Link", "DE CCGT", 
#       bus0="DE gas", 
#       bus1="DE")

## Add electricity export-import time series

In [ ]:
es.add("Load", "DE export-import",
       bus="DE", 
       p_set=export_import_load[::options["step"]].values)

## Add industry process heat demand 
Assumptions for 2050 / zero carbon. Source: IEE.2019 "“Entwicklung der Gebäudewärme und Rückkopplung mit dem Energiesystem in -95 % THG-Klimazielszenarien"
and https://ec.europa.eu/energy/sites/ener/files/documents/Report%20WP1.pdf

In [ ]:
# 2050 / zero carbon assumptions
industry = pd.Series()
industry["heat_100"] = 13   # central/urban heal sector
industry["heat_100_500"] = 56     # RH, Boiler, CHP
industry["heat_500_el"] = 145     # el. load
industry["heat_500_h2"] = 100     # H2 load
industry["heat_500_ch4"] = 170    # CH4 load, NB: this is pretty low
industry *= 1e6

### <100°C

In [ ]:
industry_heat_100_t = np.array([industry["heat_100"]/8760 ]*es.snapshots.shape[0])

es.add("Load", "DE industry <100°C heat",
       bus="DE urban heat",
       p_set=industry_heat_100_t)

### >100°C and <500°C

In [ ]:
industry_heat_100_500_t = np.array([industry["heat_100_500"]/8760]*es.snapshots.shape[0])

es.add("Bus", "DE industry >100°C heat",
       carrier="heat")

es.add("Load", "DE industry >100°C heat",
       bus="DE industry >100°C heat",
       p_set=industry_heat_100_500_t)

nodes = pd.Index(["DE"])

es.madd("Link",
        nodes + " industry resistive heater",
        bus0=nodes,
        bus1=nodes + " industry >100°C heat",
        p_nom_extendable=True,
        capital_cost=costs.at[idx['central resistive heater',2030],'efficiency']*costs.at[idx['central resistive heater',2030],'fixed'],
        efficiency=costs.at[idx['central resistive heater',2030],'efficiency'])

es.madd("Link",
        nodes + " industry gas boiler",
        bus0=nodes + " gas",
        bus1=nodes + " industry >100°C heat",
        p_nom_extendable=True,
        capital_cost=costs.at[idx['central gas boiler',2030],'efficiency']*costs.at[idx['central gas boiler',2030],'fixed'],
        efficiency=costs.at[idx['central gas boiler',2030],'efficiency'])

chp_parameters = {
    'eta_elec' : 0.468, #electrical efficiency with no heat output
    'c_v' : 0.15, #loss of fuel for each addition of heat
    'c_m' : 0.75, #backpressure ratio
    'p_nom_ratio' : 1., #ratio of max heat output to max electrical output
}

es.madd("Link",
         nodes + " industry CHP electric",
         bus0=nodes + " gas",
         bus1=nodes,
         p_nom_extendable=True,
         capital_cost=costs.at[idx['central CHP',2030],'fixed']*chp_parameters['eta_elec'],
         efficiency=chp_parameters['eta_elec'])

es.madd("Link",
         nodes + " industry CHP heat",
         bus0=nodes + " gas",
         bus1=nodes + " industry >100°C heat",
         p_nom_extendable=True,
         efficiency=chp_parameters['eta_elec']/chp_parameters['c_v'])

### heat >500°C. 

Exogenous consideration of h2 and methan demand for process heat >500°C. 

Source. K. Purr, J. Günther, H. Lehmann und P. Nuss, “Wege in eine ressourcenschonende Treibhausgasneutralität: Rescue Studie,” Dessau-Roßlau, 2019.

In [ ]:
# TODO: implement based on the study.
if True:
    es.add("Load", "DE industry PtH",
           bus="DE",
           p_set=np.array([industry["heat_500_el"]/8760]*es.snapshots.shape[0]))
    
    es.add("Load", "DE industry H2",
           bus="DE H2",
           p_set=np.array([industry["heat_500_h2"]/8760]*es.snapshots.shape[0]))

    es.add("Load", "DE industry CH4",
           bus="DE gas",
           p_set=np.array([industry["heat_500_ch4"]/8760]*es.snapshots.shape[0]))

In [ ]:
es.loads

# Change Cost, Emissions, COP

In [ ]:
def offwind_connection_costs(length_onshore = 20, length_offshore = 60):
    """
    
    calculates grid connection costs for offshore
    
    HVAC or HVDC overhead: 400 €/MW/km
    
    factor: 1.25 because it wont be a straight line
    
    offwind-ac-connection-underground: 1500 €/MW/km # NB: what about submarine?
    
    offwind-ac-station: 200 €/kW    
    
    onshore station: 80 €/kW
    
    inter-array cable: 
    """
    
    return ((400*1.25*length_onshore+1500*1.25*length_offshore+200000.+80000)*1)*(annuity(40., 0.07)+0.02)

In [ ]:
es.global_constraints.at["co2_limit","constant"] = options["co2_limit"]

es.carriers.at["gas", "co2_emissions"] = 0. # we assume synthetic/renewable gas (import)

# renewable gas import, source: ewi Energy Research & Scenarios gGmbH, “Dena-Leitstudie Integrierte Energiewende,” 2018
# Jahr 2050: "30 EUR/MWh für konventionelles Erdgas, 92 EUR/MWh für synthetisches Methan bzw. 101 EUR/MWh für synthetisches LNG,  sowie 36 EUR/MWh für konventionellen Diesel
# vs. 121 EUR/MWh für synthetischen Diesel"
es.stores.at["DE gas Store", "marginal_cost"] = 101.


# only 150000 MW of low-cost utility-scale plants; NB: 50/50 for utility/rooftop
es.generators.at["DE solar", "p_nom_max"] = 300000. 

# relax p_nom_max for onshore wind
if False: # dont do it.
    es.generators.at["DE0 onwind", "p_nom_max"] *= 1.1
    es.generators.at["DE1 onwind", "p_nom_max"] *= 1.1
    es.generators.at["DE2 onwind", "p_nom_max"] *= 1.1

# power grid expansion costs

## offwind
es.generators.at["DE offwind", "capital_cost"] = (costs.at[idx['offwind', 2030],'fixed'] 
                                                  +  offwind_connection_costs(length_onshore = 20, length_offshore = 35))

## add 200€/kW grid connection costs to onwind costs, DEA
c_on = costs.loc[idx['onwind', 2030],:]
c_on_grid = 200*1e3 * (annuity(c_on["lifetime"], c_on["discount rate"]) + c_on["FOM"]/100)

es.generators.at["DE0 onwind", "capital_cost"] = c_on["fixed"] + c_on_grid
es.generators.at["DE1 onwind", "capital_cost"] = c_on["fixed"] + c_on_grid
es.generators.at["DE2 onwind", "capital_cost"] = c_on["fixed"] + c_on_grid

c_pv_utility_grid = c_on_grid * 1.5
c_pv = (costs.loc[idx['solar-utility', 2030],"fixed"] + c_pv_utility_grid + costs.loc[idx['solar-rooftop', 2030],"fixed"])/2

# reduce heat demand for buildings. target year: 2050, moderate Sanierung. Source: IEE.2019, Entwicklung der Gebäudewärme und Rückkopplung mit dem Energiesystem in -95 % thg Klimazielszenarien

es.loads_t.p_set["DE heat"] *= 0.88
es.loads_t.p_set["DE urban heat"] *= 0.88

use_when2heat = True
if use_when2heat:
    # change heat pumps cop according to when2heat data, https://doi.org/10.1038/s41597-019-0199-y

    es.links_t.efficiency.loc[:, "DE central heat pump"] = cop_DE.loc[::8760/es.snapshots.shape[0], "ASHP"].values
    es.links_t.efficiency.loc[:, "DE ground heat pump"] = cop_DE.loc[::8760/es.snapshots.shape[0], "GSHP"].values

# Initial optimization run

In [ ]:
es.consistency_check()

In [ ]:
skip_pre=True

es.lopf(solver_name=options['solver']['name'], solver_options=options['solver']['options'], skip_pre=skip_pre,
          extra_functionality=extra_functionality)

# Prepare MGA

In [ ]:
mga_groups = {
    "generators": [["DE0 onwind", "DE1 onwind", "DE2 onwind"],
             ["DE offwind"], 
             ["DE solar", "DE solar-rooftop"],
             ["DE solar thermal collector", "DE central solar thermal collector"],
            ],
    "links": [["DE OCGT"], 
              ["DE H2 Electrolysis"],
              ["DE H2 Fuel Cell"],
              ["DE Sabatier"],
              ["DE battery charger"], 
              ["DE central heat pump"],
              ["DE ground heat pump"],
              ["DE resistive heater"],
              ["DE central resistive heater", "DE industry resistive heater"],
              ["DE gas boiler"],
              ["DE central gas boiler", "DE industry gas boiler"],
              ["DE central CHP electric", "DE industry CHP electric"]
             ],
    "stores": [["DE gas Store"]]
}

In [ ]:
mga_weights = make_mga_weights(mga_groups=mga_groups)

In [ ]:
mga_weights.tail()

In [ ]:
es.apply_mga_structure(mga_groups, mga_weights)

In [ ]:
# # Place cost optimal scenario at the end of the result summary df
es.export_to_hdf5(path = "results/results_complete_mga_iterations/iteration_opt.h5")

In [ ]:
hasattr(es.model, "mga_function_expr")

# Start mga iterations

In [ ]:
es.lopf_prepare_solver(solver_name=options['solver']['name'])


### Warning: The next code block is CPU-intensive.

In [ ]:
def solve_mga_iterations(mga_groups, mga_weights, options, 
                         slacks=[1.005, 1.01, 1.05], # slacks, e.g. [1.005, 1.01, 1.015, 1.02, 1.05, 1.1]
                         # summary,
                         # scenario_design
                        ):
    if not hasattr(es.model, "mga_function_expr"):
        raise TypeError("MGA structure must have been applied #.apply_mga_structure()")
    failed_iterations_i = [] # Save the iterations for which the solver can not find a solution. List should be empty.
    
    for slack in slacks: # slacks, e.g. [1.005, 1.01, 1.015, 1.02, 1.05, 1.1]
        for i, row in mga_weights.iterrows():

            es.model.slack = slack 
            
            for index, value in row.items():
                es.model.mga_weight[index] = value

            if options["solver"]["name"]=="gurobi_persistent":
                es.opt.set_objective(es.model.objective) # Update the objective of the persistent solver instance

                es.opt.remove_constraint(es.model.cost_budget) # Update the cost budget of the persistent solver instance
                es.opt.add_constraint(es.model.cost_budget) # Update the cost budget of the persistent solver instance

            try:
                es.lopf_solve_wo_build(solver_options=options['solver']['options'])

                es.export_to_hdf5(path = "results/results_complete_mga_iterations/iteration_"+"s"+str(s)+"w"+str(i)+".h5")

            except ValueError:
                failed_iterations_i.append(i)
                
                
        def save_scenario_design(mga_weights, slacks):
            scenario_design = pd.DataFrame()
            for s in slacks:
                scenario_design = pd.concat([scenario_design, mga_weights], ignore_index=True)
            scenario_design.insert(0, "slack", np.array([s for s in slacks for i in mga_weights.index])) 
            scenario_design.index = ["s"+str(s)+"w"+str(i) for s in slacks for i, row in mga_weights.iterrows()]
            scenario_design.to_csv("scenario_design.csv")

        save_scenario_design(mga_weights=mga_weights, slacks=slacks)
        
        return failed_iterations_i

failed_iterations_i = solve_mga_iterations(mga_groups, mga_weights, options=options)

In [ ]:
print(f"failed_iterations_i: {failed_iterations_i}")